# Mine releases from selected projects

This notebook mine the releases

In [1]:
import os
import datetime
import json
import re
import traceback
import math

from multiprocessing import Pool
import pandas as pd

from ipywidgets import IntProgress
from IPython.display import display

import releasy

In [2]:
from util import (
    DATA_PATH,
    REPO_PATH,
    TMP_PATH, 
    CPU,
    CycleType,
    RAPID_RELEASE_LIM,
    TRAD_RELEASE_LIM,
    delta2days,
    is_rapid_release,
    is_trad_release,
    get_cycle,
    get_duration,
    get_start_delay,
    get_commits_in_advance
)

In [3]:
selected_projects = pd.read_csv(DATA_PATH / '10_projects_selected.csv')

## Mine Releases


In [4]:
def mine(name: str) -> releasy.Project:
    elapsed_time = datetime.datetime.now()
    
    try:
        repo_path = str(REPO_PATH / name)
        project = releasy.Miner(repo_path, name).apply(
            releasy.FinalReleaseMiner(),
            releasy.HistoryCommitMiner(),
            releasy.BaseReleaseMiner(),
            releasy.ContributorMiner(),
            releasy.SemanticReleaseMiner()
        ).mine()

        rapid_releases = [release for release in project.main_releases if is_rapid_release(release) and len(release.commits) >= 10]
        trad_releases = [release for release in project.main_releases if is_trad_release(release) and len(release.commits) >= 10]
        
        project_data = {
            'project': project.name,
            'prefixes': len(project.releases.prefixes()),
            'prefixes_names': str(" ".join(project.releases.prefixes())),
            'main_releases': len(project.main_releases),
            'rapid_releases': len(rapid_releases),
            'trad_releases': len(trad_releases),
            'patches': len(project.patches),
            'releases': len(project.releases)
        }
        
        release_data = []
        sreleases = [release for release in project.main_releases if len(release.commits) >= 10]
        for srelease in sreleases:
            # release = srelease.release
            # cycle_days = delta2days(get_cycle(srelease))
            # delay = delta2days(srelease.delay)
            # prev_release = srelease.prev_main_release

            if not srelease.commits or not srelease.prev_main_release:
                continue
                
            cycle_days = delta2days(get_cycle(srelease))
            duration = delta2days(get_duration(srelease))
            start_delay = delta2days(get_start_delay(srelease))
            commits_in_advance = get_commits_in_advance(srelease)
            
            release_data.append({
                'project': project.name,
                'release': srelease.name,
                'previous_release': srelease.prev_main_release.name if srelease.prev_main_release else '',
                'cycle_old': delta2days(srelease.cycle),
                'cycle_days': cycle_days,
                'rapid_release': is_rapid_release(srelease),
                'trad_release': is_trad_release(srelease),
                
                'commits': len(srelease.commits),
                'commits_in_advance': len(commits_in_advance),
                'start_delay': start_delay,
                'duration': duration,
                'patches': len(srelease.patches)     
            })
        
        elapsed_time = datetime.datetime.now() - elapsed_time
        project_data['time'] = elapsed_time
    except Exception as err:
        print(f"{name:40} {err=}")
        traceback.print_exception(err)
        project_data = {}
        release_data = []
    
    return project_data, release_data

In [5]:
project_names = list(selected_projects['project'])
# project_names = project_names[:1]

In [6]:
with Pool(processes=CPU) as pool:
    processed = 0
    project_results = []
    release_results = []
    progress = IntProgress(min=0, max=len(project_names))
    display(progress)
    for result in pool.imap_unordered(mine, project_names):
        project_result, release_result = result
        project_results.append(project_result)
        release_results.extend(release_result)
        progress.value += 1


IntProgress(value=0, max=1039)

In [7]:
projects = pd.DataFrame(project_results)
projects.sample(1)

,project,prefixes,prefixes_names,main_releases,rapid_releases,trad_releases,patches,releases,time
700,swagger-api/swagger-ui,3,v/ v,69,34,10,189,258,0 days 00:00:00.340142


In [8]:
releases = pd.DataFrame(release_results)
releases.sample(10)

,project,release,previous_release,cycle_old,cycle_days,rapid_release,trad_release,commits,commits_in_advance,start_delay,duration,patches
13106,jonas/tig,tig-0.14,tig-0.13,22.925162,22.925382,True,False,78,1,-81.549421,104.474803,1
3146,garris/BackstopJS,v3.7.0,v3.6.0,9.431007,9.431007,True,False,13,0,2.736481,6.694525,5
1064,graphql-dotnet/graphql-dotnet,v0.8.0,v0.7.0,4.519282,4.519282,True,False,16,2,-258.214954,262.734236,2
25066,influxdata/influxdb,v0.10.0,v0.9.0,238.083947,238.083947,False,True,425,0,111.714132,126.369815,3
10271,HabitRPG/habitica,v4.28.0,v4.27.0,8.858310,8.858310,True,False,59,19,-4.572662,13.430972,2
16136,afollestad/material-dialogs,2.8.0,2.7.0,11.265058,11.265058,True,False,10,0,0.001181,11.263877,1
1071,graphql-dotnet/graphql-dotnet,3.0.0,v2.4.0,657.624167,657.624167,False,True,407,1,-10.042986,667.667153,0
20719,Genymobile/scrcpy,v1.23,v1.22,24.311597,24.220174,True,False,120,6,-649.197894,673.418067,0
13786,redis/redis-py,3.2.0,3.1.0,19.900336,19.900336,True,False,20,4,-40.438391,60.338727,1
5631,Zettlr/Zettlr,v1.4.0,v1.3.0,90.476238,90.476238,False,True,155,0,0.725694,89.750544,3


In [9]:
len(projects)

1039

In [10]:
len(releases)

25149

In [11]:
len(releases.query("rapid_release == True"))

13102

In [12]:
len(releases.query("trad_release == True"))

7071

In [13]:
len(releases.query("rapid_release == False and trad_release == False"))

4976

In [14]:
releases['commits'].sum()

3877182

In [15]:
projects.to_csv(DATA_PATH / '20_projects.csv', index=False)

In [16]:
releases.to_csv(DATA_PATH / '20_releases.csv', index=False)